## Libraries and versions

### Libraries

In [1]:
python_version = !Python -V #version 3.7.9
import pandas as pd #version 1.2.3
import numpy as np #version 1.19.2


#for machine learning models
#pre processing
from category_encoders import OneHotEncoder, __version__ as ce_version #version 2.2.2

#split data in train and test
#version
from sklearn import __version__ as sk_version #version 0.24.1

#pre-processing
from sklearn.model_selection import train_test_split

#classificators
from sklearn.tree import DecisionTreeClassifier

#metrics
from sklearn.metrics import accuracy_score

#explainer_dashboard
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

### Versions

In [2]:
print(f'{python_version[0]}')
print(f'Pandas version: {pd.__version__}')
print(f'Numpy version: {np.__version__}')
print(f'Category Encoders version: {ce_version}')
print(f'Sklearn version: {sk_version}')

Python 3.7.9
Pandas version: 1.2.4
Numpy version: 1.20.1
Category Encoders version: 2.2.2
Sklearn version: 0.24.1


## Configurations

In [3]:
import warnings
warnings.filterwarnings("ignore")

## Dataset

In [4]:
dataset = pd.read_csv('train_cleaning_model7.csv')

In [5]:
dataset.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,age_categories
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D,18-25
1,462643,Female,Yes,38,Yes,Engineer,1.0,Average,3.0,Cat_4,A,36-40
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B,66-70
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B,66-70
4,462669,Female,Yes,40,Yes,Entertainment,1.0,High,6.0,Cat_6,A,36-40


# Dataset transformation

## Encoders

### OneHotEncoder

In [6]:
#OneHotEncoder
encoder = OneHotEncoder(cols=['Ever_Married', 'Graduated', 'Profession', 'Spending_Score'],
                       use_cat_names=True)
dataset_onehorencoder = encoder.fit_transform(dataset)

#map column Segmentation and transform for numeric variables
map_segmentation = {'A':0, 'B':1, 'C':2, 'D':3}
dataset_onehorencoder['Segmentation'] = dataset_onehorencoder['Segmentation'].map(map_segmentation)

#drop non used columns ID, Gender and Family_Size
dataset_onehorencoder.drop(columns=['ID','Gender', 'Family_Size', 'age_categories', 'Var_1'], inplace=True)

## Variables

### OneHotEncoder

In [7]:
#selection of all variables, except Segmentation
X_onehotencoder = dataset_onehorencoder.loc[:, dataset_onehorencoder.columns != 'Segmentation']
y_onehotencoder = dataset_onehorencoder['Segmentation']

In [8]:
#first - split train and test data
accuracy_list = []
iterations = 50
for iter in range(0, iterations):
    X_train, X_test, y_train, y_test = train_test_split(X_onehotencoder, y_onehotencoder, train_size=0.7,
                                                        stratify=y_onehotencoder)

    #third - macinhe learning apply
    model7_1 = DecisionTreeClassifier(max_depth=5,
                                      min_samples_leaf=30,
                                      criterion='entropy',
                                      class_weight={0:0.24, 1:0.23, 2:0.24, 3:0.27})
    model7_1.fit(X_train, y_train)
    y_predict = model7_1.predict(X_test)

    #fourth - test accuray
    accuracy = accuracy_score(y_test, y_predict)
    accuracy_list.append(accuracy)

max_accuracy = np.asarray(accuracy_list).max()
mean_accuracy = np.asarray(accuracy_list).mean()
min_accuracy = np.asarray(accuracy_list).min()
std_accuracy = np.asarray(accuracy_list).std()
print(f'ACCURACY FOR {iterations} ITERATIONS')
print(f'Max = {max_accuracy:.4f}\nMean = {mean_accuracy:.4f}\nMin = {min_accuracy:.4f}\nStd = {std_accuracy:.4f}')
print(model7_1.set_params())

ACCURACY FOR 50 ITERATIONS
Max = 0.5411
Mean = 0.5231
Min = 0.5085
Std = 0.0083
DecisionTreeClassifier(class_weight={0: 0.24, 1: 0.23, 2: 0.24, 3: 0.27},
                       criterion='entropy', max_depth=5, min_samples_leaf=30)


In [9]:
#create DataFrame with errors
dict_compare = {'Real':y_test.values, 'Predict':y_predict}
dataset_error = pd.DataFrame(dict_compare, index=y_test.index)
dataset_error['Error'] = dataset_error['Real'] !=dataset_error['Predict']
dataset_error['Cat_Real'] = dataset_error['Real'].map({0:'A', 1:'B', 2:'C', 3:'D'})
dataset_error['Cat_Predict'] = dataset_error['Predict'].map({0:'A', 1:'B', 2:'C', 3:'D'})

In [10]:
print('Percent of Error for each Segmentation')
pd.crosstab(index=dataset_error['Cat_Real'], columns=dataset_error['Error'], normalize='index')

Percent of Error for each Segmentation


Error,False,True
Cat_Real,,
A,0.471429,0.528571
B,0.292135,0.707865
C,0.528998,0.471002
D,0.714734,0.285266


In [11]:
importance = model7_1.feature_importances_
pd.DataFrame(data={'Importance':importance}, index=X_onehotencoder.columns).sort_values('Importance', ascending=False)

,Importance
Age,0.508193
Spending_Score_Low,0.184064
Profession_Artist,0.161188
Profession_Healthcare,0.066269
Graduated_No,0.036271
Profession_Entertainment,0.013539
Profession_Marketing,0.010563
Work_Experience,0.009385
Spending_Score_Average,0.004493
Ever_Married_Yes,0.003912


## Explainer Dashboard

In [12]:
explainer = ClassifierExplainer(model7_1, X_train, y_train)

Note: model_output=='probability', so assuming that raw shap output of DecisionTreeClassifier is in probability space...
Generating self.shap_explainer = shap.TreeExplainer(model)


In [13]:
ExplainerDashboard(explainer).run()

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating roc auc curves...
Calculating prediction probabilities...
Calculating liftcurve_dfs...
Calculating predictions...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating pred_percentiles...
Calculating pr auc curves...
Calculating metrics...
Calculating shap interaction values... (this may take a while)
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation

 * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
192.168.0.247 - - [29/Apr/2021 10:45:36] "GET / HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:36] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:36] "GET /_dash-layout HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.247 - - [29/Apr/2021 10:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.247 - - [29/Ap